In [7]:
import pandas as pd
import numpy as np


In [12]:
def plaintxt(text,nospace=True): #removes all spaces, line breaks, and punctuation from text
    removees = ["\n",".",",","?","(",")","!","/",":",";","-","'",'"',"[","]"]
    if nospace: # If nospace is false, spaces will not be removed
        removees.append(" ")
    return remover(text,removees)

In [21]:
def phrases(text,n):
    nopunc = plaintxt(text,nospace=False) #probably want to add something here to deal with sentences ending, but
                                          # I'll do that later
    words = nopunc.lower().split()
    phraselist = [" ".join(words[i:i+n]) for i in range(len(words)-n+1)] #I might need to change " " to "" when I 
                                            # use this with the word vectorizer
    return phraselist


def remover(text,removees): #removes a list of strings from text
    nothing = [ "" for i in removees ]
    return mreplace(text,removees,nothing)

def mreplace(text,list1,list2): #replaces strings in text more efficiently than built-in methods
    for i in range(len(list1)):
        text=text.replace(list1[i],list2[i])
    return text

In [22]:
def pseudoPCA(df,n,numfeatures): 
    speechphrases = [phrases(text,n) for text in df["Transcript"]]
    allphrases = []
    for i in speechphrases:
        allphrases += i
    phraselist = np.unique(allphrases)
    newdf = df.copy()
    labels = df["President"]
    presidents = np.unique(labels)
    stds = []
    phrasestds = {}
    for phrase in phraselist:
        newdf[phrase] = newdf["Transcript"].apply(lambda text: phrases(text,n).count(phrase))
        aves = []
        for pres in presidents:
            rows = newdf[newdf["President"]==pres] # pick out only the rows corresponding to each president
            aves.append(sum(rows[phrase])/len(rows[phrase])) #avereage number of times each pres uses the phrase
        mean = sum(aves)/len(aves)
        devs = np.array(aves)-mean
        var = sum(devs**2)/len(devs)
        std = var**0.5 # We calculate the standard deviation of the presidents' average no of phrase uses
        stds.append(std)
        phrasestds[phrase] = std
    stds.sort(reverse=True)
    cutoff = stds[n-1] #Only the phrases in the top n std devs will be picked as features 
    for phrase in phraselist:
        if phrasestds[phrase] < cutoff:
            newdf.drop(phrase,axis=1) # get rid of features with std dev too small - they are less significant
    features = newdf.drop(["President","Transcript"],axis=1)
    return features

In [23]:
orig_data = pd.read_csv("archive/presidential_speeches.csv")

names = ['George Washington', 'Donald Trump']
data = orig_data[orig_data['President'].isin(names)]
data = data.reset_index(drop = True)

In [24]:
features2 = pseudoPCA(data,2,5)
features2

KeyboardInterrupt: 